In [2]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split
from geopy.geocoders import Nominatim
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
color = sns.color_palette()
pd.options.mode.chained_assignment = None  # default='warn'

In [4]:
df1 = pd.read_excel("2019_bronx.xlsx")
df2 = pd.read_excel("2019_brooklyn.xlsx")
df3 = pd.read_excel("2019_manhattan.xlsx")
df4 = pd.read_excel("2019_queens.xlsx")
df5 = pd.read_excel("2019_statenisland.xlsx")

In [6]:
df1['AREA'] = 'bronx'
df2['AREA'] = 'brooklyn'
df3['AREA'] = 'manhattan'
df4['AREA'] = 'queens'
df5['AREA'] = 'statenisland'


In [8]:
frames = [df1, df2, df3, df4, df5]
df = pd.concat(frames)

In [9]:
df.columns = ['BOROUGH',
                  'NEIGHBORHOOD',
                  'BUILDING CLASS CATEGORY',
                  'TAX CLASS AS OF FINAL ROLL 18/19',
                  'BLOCK',
                  'LOT',
                  'EASE-MENT',
                  'BUILDING CLASS AS OF FINAL ROLL 18/19',
                  'ADDRESS',
                  'APARTMENT NUMBER',
                  'ZIP CODE',
                  'RESIDENTIAL UNITS',
                  'COMMERCIAL UNITS',
                  'TOTAL UNITS',
                  'LAND SQUARE FEET',
                  'GROSS SQUARE FEET',
                  'YEAR BUILT',
                  'TAX CLASS AT TIME OF SALE',
                  'BUILDING CLASS AT TIME OF SALE',
                  'SALE PRICE',
                  'SALE DATE',
                  'AREA']

In [10]:
df['FINAL_ADDRESS'] = df['ADDRESS'].apply(lambda x: x + " NYC")

In [11]:
df = df.drop(['EASE-MENT', 'APARTMENT NUMBER'], axis=1)

df = df[df['SALE PRICE'] < 3e6]
df = df[df['SALE PRICE'] > 5000]

df = df[df['RESIDENTIAL UNITS'] < 10.0]

df = df[df['COMMERCIAL UNITS'] < 5.0]

df = df[df['YEAR BUILT'] > 1860]
df = df[df['YEAR BUILT'] <= 2020]

df = df[df['GROSS SQUARE FEET'] < 4600]
df = df[df['GROSS SQUARE FEET'] > 50]

df = df[df['LAND SQUARE FEET'] > 200]
df = df[df['LAND SQUARE FEET'] < 7000]

llimit = np.percentile(df['ZIP CODE'], 1)
df = df[df['ZIP CODE'] > llimit]

df = df[df['TOTAL UNITS'] < 5]
df = df[df['TOTAL UNITS'] > 0]

df = df[df['LOT'] < 250]

In [12]:
df = df.dropna(how='any', axis=0)

In [14]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)
train_target = df_train['SALE PRICE']
train = df_train.drop(['SALE PRICE'], axis=1)
test_target = df_test['SALE PRICE']
test = df_test.drop(['SALE PRICE'], axis=1)

In [15]:
cat = ['NEIGHBORHOOD', 'BUILDING CLASS CATEGORY', 'BUILDING CLASS AS OF FINAL ROLL 18/19', 'ADDRESS',
           'BUILDING CLASS AT TIME OF SALE', 'SALE DATE', 'AREA', 'FINAL_ADDRESS']
final = pd.concat([train, test])
for col in cat:
    lb = LabelEncoder()
    final[col] = lb.fit_transform(final[col].values)

final['TAX CLASS AS OF FINAL ROLL 18/19'] = final['TAX CLASS AS OF FINAL ROLL 18/19']\
    .map({'2A': 2, '1': 1, '4': 4})

In [16]:
for col in final.columns:
    minmax = MinMaxScaler()
    final[col] = minmax.fit_transform(final[col].values.reshape(-1, 1))

In [17]:
train = final[:len(train)]
test = final[len(train):]

In [32]:
rf = RandomForestRegressor(n_estimators=300, verbose=True, max_depth=10, n_jobs=-1)
rf.fit(train, train_target)
res_pred = rf.predict(test)
rms = np.sqrt(mean_squared_error(test_target, res_pred))
print("RMS: %f" % rms)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   13.1s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:   20.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 300 out of 300 | elapsed:    0.1s finished


RMS: 282412.346328


In [22]:
train.head(1)

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AS OF FINAL ROLL 18/19,BLOCK,LOT,BUILDING CLASS AS OF FINAL ROLL 18/19,ADDRESS,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE DATE,AREA,FINAL_ADDRESS
20590,0.666667,0.76,0.4375,1.0,0.570736,0.03629,0.561404,0.928248,0.801724,0.0,0.25,0.0,0.510235,0.752051,0.409396,1.0,0.568966,0.82866,0.666667,0.928248


In [21]:
print(rf.feature_importances_)

[0.00482569 0.05207141 0.00559672 0.0010736  0.28450213 0.01411966
 0.0105638  0.0103002  0.19780994 0.00392554 0.00339836 0.00334931
 0.05258587 0.29033312 0.02547732 0.00089173 0.00846692 0.01507589
 0.00531113 0.01032166]


In [31]:
lgbm = lgb.LGBMRegressor(max_depth=15, num_leaves=40)
lgbm.fit(train, train_target)
res_pred = lgbm.predict(test)
rms = np.sqrt(mean_squared_error(test_target, res_pred))
print("RMS: %f" % rms)

RMS: 272372.432552


In [23]:
print(lgbm.feature_importances_)

[ 66 425  64   8 647 220 157 321 491  42   6  20 427 457 267   0  29 253
   0   0]


In [20]:
xg_reg = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.3, learning_rate=0.1,
                              max_depth=5, alpha=10, n_estimators=10)
xg_reg.fit(train, train_target)
res_pred = xg_reg.predict(test)
rms = np.sqrt(mean_squared_error(test_target, res_pred))
print("RMS: %f" % rms)

RMS: 441818.416454


In [24]:
print(xg_reg.feature_importances_)

[0.13290177 0.03749656 0.02913029 0.0076951  0.06572558 0.00531941
 0.15653601 0.00484527 0.16456841 0.0132339  0.00590178 0.
 0.02618903 0.11737117 0.03847794 0.00490686 0.1053235  0.00450877
 0.07310537 0.00676332]
